# Enter Statefarm competition

In [1]:
from theano.sandbox import cuda
cuda.use('gpu1')

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
WARNING (theano.sandbox.cuda): Ignoring call to use(1), GPU number 0 is already in use.


In [30]:
%cd /home/ubuntu/nbs/solutions/statefarm
%matplotlib inline
from __future__ import print_function, division
import utils; reload(utils)
from utils import *
from IPython.display import FileLink
import pandas as pd

/home/ubuntu/nbs/solutions/statefarm


Characterize data set first

In [31]:
%cd /home/ubuntu/data/statefarm

/home/ubuntu/data/statefarm


In [32]:
%ls -1 train/*/*.jpg | wc

  22424   22424  782853


In [33]:
%ls -1 test/unknown | wc

  79726   79726 2065911


In [34]:
df = pd.read_csv('driver_imgs_list.csv')

In [44]:
print(df.head())

  subject classname            img
0    p002        c0  img_44733.jpg
1    p002        c0  img_72999.jpg
2    p002        c0  img_25094.jpg
3    p002        c0  img_69092.jpg
4    p002        c0  img_92629.jpg


In [48]:
print(df.groupby(['subject']).count())

         classname   img
subject                 
p002           725   725
p012           823   823
p014           876   876
p015           875   875
p016          1078  1078
p021          1237  1237
p022          1233  1233
p024          1226  1226
p026          1196  1196
p035           848   848
p039           651   651
p041           605   605
p042           591   591
p045           724   724
p047           835   835
p049          1011  1011
p050           790   790
p051           920   920
p052           740   740
p056           794   794
p061           809   809
p064           820   820
p066          1034  1034
p072           346   346
p075           814   814
p081           823   823


How to split test and validation sets based on subject/driver
There are n = 26 drivers and approximately 22K images.
~ 10% of the data for validation is data for 2-3 drivers
Move data for 3 drivers picked at random p002, p049, p061

In [52]:
subjects = np.random.permutation(df.ix[:,'subject'].unique())
print(subjects[:3])

['p072' 'p016' 'p012']


In [65]:
print(df[df['subject'].isin(subjects[:3])].ix[:,['classname', 'img']].head())

    classname            img
725        c0  img_10206.jpg
726        c0  img_27079.jpg
727        c0  img_50749.jpg
728        c0  img_97089.jpg
729        c0  img_37741.jpg


In [66]:
%cd train

/home/ubuntu/data/statefarm/train


In [68]:
%mkdir ../valid

In [69]:
for d in glob('*'):
    print(d)
    os.mkdir('../valid/' + d)

c7
c8
c6
c0
c2
c4
c9
c1
c3
c5


In [84]:
val_imgs = df[df['subject'].isin(subjects[:3])].ix[:,['img']].values
val_cls = df[df['subject'].isin(subjects[:3])].ix[:,['classname']].values
print(len(val_imgs))
print(len(val_cls))
print(val_imgs.item(1))

2247
2247
img_27079.jpg


In [87]:
for i in range(2247):
    #print(val_cls.item(i))
    os.rename(val_cls.item(i) + '/' + val_imgs.item(i), '../valid/' + val_cls.item(i) + '/' + val_imgs.item(i))

### Create sample

In [89]:
from shutil import copyfile

In [90]:
%mkdir ../sample
%mkdir ../sample/train
%mkdir ../sample/valid

In [91]:
for d in glob('*'):
    print(d)
    os.mkdir('../sample/valid/'+d)
    os.mkdir('../sample/train/'+d)

c7
c8
c6
c0
c2
c4
c9
c1
c3
c5


In [98]:
%cd /home/ubuntu/data/statefarm/train
g = glob('*/*.jpg')
shuf = np.random.permutation(g)
for i in range(3000): copyfile(shuf[i], '../sample/train/'+shuf[i])

/home/ubuntu/data/statefarm/train


In [99]:
%cd ../valid
g = glob('*/*.jpg')
shuf = np.random.permutation(g)
for i in range(300): copyfile(shuf[i], '../sample/valid/'+shuf[i])

/home/ubuntu/data/statefarm/valid


### Create batches

In [100]:
%cd /home/ubuntu/nbs/solutions/statefarm

/home/ubuntu/nbs/solutions/statefarm


In [101]:
batch_size = 64
#path = '/home/ubuntu/data/statefarm/'
path = '/home/ubuntu/data/statefarm/sample/'
batches = get_batches(path+'train', batch_size=batch_size)

Found 3000 images belonging to 10 classes.


In [102]:
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

Found 300 images belonging to 10 classes.


In [104]:
test_batches = get_batches('/home/ubuntu/data/statefarm/'+'test', batch_size=batch_size*2, shuffle=False)

Found 79726 images belonging to 1 classes.


#### new VGG16 has batchnorm layers

- use pre-trained VGG weights
- precompute the output of the last convolution layer

In [105]:
vgg = Vgg16()
model = vgg.model
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]

553476096/553620808 [============================>.] - ETA: 0s

In [108]:
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, trn_filenames, test_filenames) = get_classes(path)

Found 3000 images belonging to 10 classes.
Found 300 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [109]:
conv_model = Sequential(conv_layers)
conv_feat = conv_model.predict_generator(batches, batches.nb_sample)
conv_val_feat = conv_model.predict_generator(val_batches, val_batches.nb_sample)
conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)

In [110]:
%mkdir /home/ubuntu/data/statefarm/sample/results

OSError: [Errno 12] Cannot allocate memory

In [112]:
save_array(path+'results/conv_val_feat.dat', conv_val_feat)
save_array(path+'results/conv_test_feat.dat', conv_test_feat)
save_array(path+'results/conv_feat.dat', conv_feat)

In [113]:
conv_val_feat.shape

(300, 512, 14, 14)

#### Batchnorm dense layers on pretrained conv layers

In [123]:
def get_maxpool_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p/2),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]

In [124]:
def get_avgpool_bn_layers(p):
    return [
        AveragePooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p/2),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]

In [125]:
p = 0.8

In [126]:
bn_mp_model = Sequential(get_maxpool_bn_layers(p))

In [127]:
bn_mp_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [128]:
bn_mp_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=10, 
             validation_data=(conv_val_feat, val_labels))

Train on 3000 samples, validate on 300 samples
Epoch 1/10
3000/3000 [==============================] - 1s - loss: 4.9338 - acc: 0.0937 - val_loss: 3.2086 - val_acc: 0.0967
Epoch 2/10
3000/3000 [==============================] - 1s - loss: 4.1513 - acc: 0.1117 - val_loss: 2.9386 - val_acc: 0.0600
Epoch 3/10
3000/3000 [==============================] - 1s - loss: 3.7267 - acc: 0.1100 - val_loss: 2.6660 - val_acc: 0.1267
Epoch 4/10
3000/3000 [==============================] - 1s - loss: 3.3693 - acc: 0.1203 - val_loss: 2.4170 - val_acc: 0.1133
Epoch 5/10
3000/3000 [==============================] - 1s - loss: 3.1525 - acc: 0.1287 - val_loss: 2.4636 - val_acc: 0.1033
Epoch 6/10
3000/3000 [==============================] - 1s - loss: 2.9110 - acc: 0.1333 - val_loss: 2.4147 - val_acc: 0.1500
Epoch 7/10
3000/3000 [==============================] - 1s - loss: 2.7673 - acc: 0.1380 - val_loss: 2.3964 - val_acc: 0.1333
Epoch 8/10
3000/3000 [==============================] - 1s - loss: 2.6551 - ac

In [130]:
bn_ap_model = Sequential(get_avgpool_bn_layers(p))
bn_ap_model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [131]:
bn_ap_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=10, 
             validation_data=(conv_val_feat, val_labels))

Train on 3000 samples, validate on 300 samples
Epoch 1/10
3000/3000 [==============================] - 1s - loss: 5.1889 - acc: 0.0957 - val_loss: 3.2649 - val_acc: 0.0500
Epoch 2/10
3000/3000 [==============================] - 1s - loss: 4.9104 - acc: 0.1100 - val_loss: 2.8649 - val_acc: 0.0567
Epoch 3/10
3000/3000 [==============================] - 1s - loss: 4.7459 - acc: 0.1170 - val_loss: 2.6780 - val_acc: 0.0867
Epoch 4/10
3000/3000 [==============================] - 1s - loss: 4.5417 - acc: 0.1270 - val_loss: 2.7491 - val_acc: 0.0933
Epoch 5/10
3000/3000 [==============================] - 1s - loss: 4.4365 - acc: 0.1273 - val_loss: 2.6872 - val_acc: 0.0800
Epoch 6/10
3000/3000 [==============================] - 1s - loss: 4.2478 - acc: 0.1370 - val_loss: 2.6552 - val_acc: 0.0700
Epoch 7/10
3000/3000 [==============================] - 1s - loss: 4.2960 - acc: 0.1340 - val_loss: 2.8839 - val_acc: 0.0567
Epoch 8/10
3000/3000 [==============================] - 1s - loss: 4.1476 - ac

#### Data augmentation + dropout

In [132]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
da_batches = get_batches(path+'train', gen_t, batch_size=batch_size, shuffle=False)

Found 3000 images belonging to 10 classes.


In [133]:
da_conv_feat = conv_model.predict_generator(da_batches, da_batches.nb_sample*5)
save_array(path+'results/da_conv_feat.dat', da_conv_feat)

Combine real and augmented training data

In [134]:
da_conv_feat = np.concatenate([da_conv_feat, conv_feat])
da_trn_labels = np.concatenate([trn_labels]*6)

In [138]:
bn_mp_da_model = Sequential(get_maxpool_bn_layers(p))
bn_mp_da_model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
bn_ap_da_model = Sequential(get_avgpool_bn_layers(p))
bn_ap_da_model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [139]:
bn_mp_da_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=10, 
             validation_data=(conv_val_feat, val_labels))

Train on 18000 samples, validate on 300 samples
Epoch 1/10
18000/18000 [==============================] - 7s - loss: 4.0269 - acc: 0.1834 - val_loss: 1.7636 - val_acc: 0.4100
Epoch 2/10
18000/18000 [==============================] - 7s - loss: 2.7181 - acc: 0.3461 - val_loss: 1.4227 - val_acc: 0.5567
Epoch 3/10
18000/18000 [==============================] - 7s - loss: 2.0240 - acc: 0.4749 - val_loss: 1.3627 - val_acc: 0.5133
Epoch 4/10
18000/18000 [==============================] - 7s - loss: 1.6477 - acc: 0.5576 - val_loss: 1.3945 - val_acc: 0.5133
Epoch 5/10
18000/18000 [==============================] - 7s - loss: 1.3754 - acc: 0.6225 - val_loss: 1.4467 - val_acc: 0.5067
Epoch 6/10
18000/18000 [==============================] - 7s - loss: 1.1526 - acc: 0.6687 - val_loss: 1.4491 - val_acc: 0.5633
Epoch 7/10
18000/18000 [==============================] - 7s - loss: 1.0307 - acc: 0.7038 - val_loss: 1.5372 - val_acc: 0.5233
Epoch 8/10
18000/18000 [==============================] - 7s - 

In [140]:
bn_ap_da_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))

Train on 18000 samples, validate on 300 samples
Epoch 1/4
18000/18000 [==============================] - 8s - loss: 3.8326 - acc: 0.2282 - val_loss: 1.4204 - val_acc: 0.5700
Epoch 2/4
18000/18000 [==============================] - 8s - loss: 2.4322 - acc: 0.4383 - val_loss: 1.2023 - val_acc: 0.6433
Epoch 3/4
18000/18000 [==============================] - 8s - loss: 1.7747 - acc: 0.5632 - val_loss: 1.1518 - val_acc: 0.6600
Epoch 4/4
18000/18000 [==============================] - 8s - loss: 1.4019 - acc: 0.6383 - val_loss: 1.2723 - val_acc: 0.6167


In [141]:
### Test and submit preds

In [142]:
ap_preds = bn_ap_da_model.predict(conv_test_feat, batch_size=batch_size*2)

In [143]:
test_filenames[1:10]

['unknown/img_14887.jpg',
 'unknown/img_62885.jpg',
 'unknown/img_45125.jpg',
 'unknown/img_22633.jpg',
 'unknown/img_45660.jpg',
 'unknown/img_88962.jpg',
 'unknown/img_92987.jpg',
 'unknown/img_26932.jpg',
 'unknown/img_57972.jpg']

In [146]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)
classes

['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

In [147]:
import pandas as pd
ap_submit = pd.DataFrame(ap_preds, columns=classes)
ap_submit.insert(0, 'img', [a[8:] for a in test_filenames])

In [148]:
ap_submit.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_81601.jpg,0.143158,0.134245,0.050120,0.051986,0.102335,0.036187,0.052485,0.187010,0.130746,0.111729
1,img_14887.jpg,0.310704,0.071811,0.017291,0.029031,0.063613,0.048500,0.016212,0.043454,0.268913,0.130471
2,img_62885.jpg,0.050943,0.051950,0.127931,0.212589,0.316525,0.053531,0.029112,0.011644,0.106410,0.039365
3,img_45125.jpg,0.017157,0.039254,0.090616,0.011537,0.040201,0.002216,0.054886,0.120770,0.579970,0.043393
4,img_22633.jpg,0.197044,0.193440,0.062387,0.020345,0.055224,0.073637,0.030071,0.105970,0.172458,0.089423


In [157]:
ap_submit_name = path+'results/ap_submit2.csv'
ap_submit.to_csv(ap_submit_name, index=False)
print(path)

/home/ubuntu/data/statefarm/sample/


In [158]:
FileLink('../../../data/statefarm/sample/results/ap_submit2.csv')

/home/ubuntu/data/statefarm/sample/results/ap_submit2.csv